In [ ]:
from crossref.restful import Works
import re
import pandas as pd
import time
import requests

from citeproc import CitationStylesStyle, CitationStylesBibliography
from citeproc import Citation, CitationItem
from citeproc import formatter
from citeproc.source.json import CiteProcJSON
from tqdm import tqdm

: 

In [2]:
bibnumber_pattern = "^([0-9]+\.|\[[0-9]+\]|\(.+\))\s+"
delay = 0.5

In [3]:
input_file_path = "./testdata.txt"
bibliography_lines = []
bibnumbers = []
with open(input_file_path,"r") as f:
    for l in f:
        bibnumber = re.search(bibnumber_pattern,l).groups()
        assert len(bibnumber) <= 1
        bibnumber = bibnumber[0] if len(bibnumber) == 1 else ""
        bibnumbers.append(bibnumber)
        bibliography_lines.append(l.replace(bibnumber,"").strip())
bibliography_df = pd.DataFrame({"bibnumber":bibnumbers,"bibliography":bibliography_lines})

In [4]:
bibliography_df.head()

,bibnumber,bibliography
0,1.,Barresi M J F and Gilbert S F (2019) Developme...
1,2.,Ladoux B and Mège R-M (2017) Mechanobiology of...
2,3.,"Hollandi R, Moshkov N, Paavolainen L, Tasnadi ..."
3,4.,"Maška M, Ulman V, Delgado-Rodriguez P, Gómez-d..."
4,5.,Anlaş A A and Nelson C M (2018) Tissue mechani...


In [5]:
works = Works()

In [6]:
for line in tqdm(bibliography_lines[3:4]):
    query = works.query(bibliographic=line).sort('relevance').select("DOI")
    #query.custom_header.update({"Accept": "application/vnd.citationstyles.csl+json"})
    res = [r for _,r in zip(range(2),query)]
    time.sleep(delay)

100%|██████████| 1/1 [00:01<00:00,  1.79s/it]


In [7]:
line

'Maška M, Ulman V, Delgado-Rodriguez P, Gómez-de-Mariscal E, Nečasová T, Guerrero Peña FA, Ren TI, Meyerowitz EM, Scherr T, Löffler K, and Mikut R (2023) The cell tracking challenge: 10 years of objective benchmarking. Nat. Methods 20: 1–11.'

In [8]:
res

[{'DOI': '10.1038/s41592-023-01879-y'}, {'DOI': '10.1101/2021.06.26.450019'}]

In [9]:
r = res[0]
headers = {
    "Accept": "application/vnd.citationstyles.csl+json"
}

csl_json_data = []
for r in tqdm(res):
    d = requests.get(f"http://doi.org/{r['DOI']}", headers = headers)
    if d.status_code == 200:
        csl_json_data.append(d.json())
    time.sleep(delay)

100%|██████████| 2/2 [00:03<00:00,  1.86s/it]


In [10]:
import json
with open("testdata-csl.json","w") as f:
    json.dump(csl_json_data,f)

In [13]:
d["type"]="article-journal"

In [15]:
for d in csl_json_data:
    d["id"] = d["DOI"]
bib_source = CiteProcJSON(csl_json_data)
bib_style2 = CitationStylesStyle("./microscopy.csl", validate=False)
bibliography = CitationStylesBibliography(bib_style2, bib_source, formatter.plain)

c = Citation([CitationItem(csl_json_data[0]["id"])])
bibliography.register(c)
bibliography.cite(c, lambda x: print(x))
for item in bibliography.bibliography():
    print(str(item))

1. Maška M, Ulman V, Delgado-Rodriguez P, Gómez-de-Mariscal E, Nečasová T, Guerrero Peña F A, Ren T I, Meyerowitz E M, Scherr T, Löffler K, et al. (2023) The Cell Tracking Challenge: 10 years of objective benchmarking. Nat Methods 20: 1010–1020
